# Programming Exercise 4: Neural Network Learning

## Giới thiệu

Trong bài tập này, chúng ta sẽ cài đặt thuật toán **backpropagation** cho mạng neural và áp dụng vào bài toán nhận dạng chữ số viết tay. Dữ liệu dùng cho bài tập này gồm hai file:

- *ex4data1.mat* là dữ liệu huấn luyện cho bài toán nhận dạng chữ số viết tay.
- *ex4weights.mat* lưu trữ các tham số của neural networks.

Bạn có thể xem cài đặt chi tiết của bài tập này trong một script duy nhất tại [đây](https://github.com/minhpqn/Machine-Learning-Dojo/blob/master/code/neural_networks_learning/Neural_Networks_Learning.py).

## 1. Neural Networks

Trong [bài tập trước](https://github.com/minhpqn/Machine-Learning-Dojo/blob/master/code/multi_class_nn/Multi-class_NN.ipynb), chúng ta đã cài đặt thuật toán **feedforward propagation** cho mạng neural và sử dụng nó để nhận dạng chữ số viết tay với các tham số được cho trước. Trong bài tập này, chúng ta sẽ cài đặt thuật toán **backpropagation** để học các tham số cho mạng neural.

### 1.1 Visualizing the data

File dữ liệu ```ex4data1.mat``` có 5000 ví dụ về chữ số viết tay. File này được lưu ở định dạng ```.mat``` của Octave/Matlab. Để đọc dữ liệu trong python, ta sẽ sử dụng hàm ```scipy.io.loadmat``` (Tham khảo thêm tại [scipy.io](http://docs.scipy.org/doc/scipy/reference/io.html)).

In [1]:
%matplotlib inline
import sys
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import loadmat

mat_dict = loadmat('ex4data1.mat')
X = mat_dict['X']
y = mat_dict['y']
m = X.shape[0]

sys.stdout.write('Shape of X: ')
print X.shape
sys.stdout.write('Shape of y: ')
print y.shape

# Flatten y to simplify computation, now y has shape (5000,) and 1D
y = y.flatten()

Shape of X: (5000, 400)
Shape of y: (5000, 1)


Đoạn code trên đọc dữ liệu vào các biến ```X``` và ```y``` kiểu ```numpy.ndarray```. Trong đó, ```y``` chứa các nhãn của các example trong dữ liệu. Vì dữ liệu được biến đổi thành dạng tương thích với Octave/Matlab nên chữ số "0" sẽ được biểu diễn bằng nhãn "10" trong tập dữ liệu.

Mỗi ảnh trong tập dữ liệu được lưu dưới dạng 20x20, nên mỗi example là một vector với số chiều là 400. Nếu ta coi mỗi vector là các vector cột thì ```X``` sẽ được biểu diễn dưới dạng sau đây.

$$X=\left[
\begin{array}{c}
-\left(x^{(1)}\right)^T-\\
-\left(x^{(2)}\right)^T-\\
\vdots\\
-\left(x^{(m)}\right)^T-\\
\end{array}
\right]$$

Chúng ta sẽ bắt đầu bằng việc visualize dữ liệu. Trong phần này, chúng ta sẽ chọn ngẫu nhiên 100 examples từ dữ liệu và hiển thị các số tương ứng dưới dạng ảnh pixel 20x20 nền xám.

*Note*: Vì đây không phải là nội dung chính của bài học nên tạm thời tôi sẽ bỏ qua phần này để làm các phần quan trọng trước.

In [2]:
print 'Visualizing Data ...'

Visualizing Data ...


### 1.2 Model representation

Hình 2 thể hiện mạng neural chúng ta sử dụng. Mạng neural này bao gồm 3 tầng: tầng input, 1 tầng ẩn, và 1 tầng output.

Các tham số của mạng neural $(\Theta_{(1)},\Theta_{(2)})$ được cung cấp sẵn và lưu trong file ```ex3weights.mat```. Các tham số này có các chiều tương thích với số units của mỗi tầng. Ở đây, chúng ta có 25 units ở tầng thứ 2 (tầng ẩn) và 10 units ở tầng output (tương ứng với 10 digit classes).

<img src="neural_network_model.png" alt="Drawing" style="width:350px;"/>

Đoạn code dưới đây sẽ đọc vào dữ liệu và các tham số từ file.

In [3]:
from scipy.io import loadmat

para_dict = loadmat('ex4weights.mat')
Theta1 = para_dict['Theta1']
Theta2 = para_dict['Theta2']

print('Shape of Theta1 and Theta2:')
print Theta1.shape
print Theta2.shape

Shape of Theta1 and Theta2:
(25, 401)
(10, 26)


### 1.3 Feedforward và cost function

Chúng ta sẽ cài đặt hàm cost và gradient cho mạng neural. Như chúng ta đã học trong [bài giảng](https://github.com/minhpqn/Machine-Learning-Dojo/blob/master/code/neural_networks_learning/Lecture9.pdf), hàm cost cho mạng neural (chưa sử dụng regularization) được tính như sau.

$$J(\theta)=\frac{1}{m}\sum_{i=1}^{m}\sum_{k=1}^{K}\left[-y_k^{(i)}\log((h_\theta(x^{(i)}))_k)-(1-y_k^{(i)}) \log(1-(h_\theta(x^{(i)}))_k) \right]$$

Trong đó $h_\theta(x^{(i)})$ được tính như trong hình 2, $K=10$ là tổng số labels trong dữ liệu. Chú ý rằng $h_\theta(x^{(i)})_k=a_k^{(3)}$ là *activation* (đầu ra) của unit thứ $k$ của tầng output.

Măc dù các nhãn ban đầu của dữ liệu là 1, 2, ..., 10, khi huấn luyện mạng neural, chúng ta sẽ biểu diễn các nhãn bằng các vector chỉ gồm giá trị 0 và 1.

$$y=
\left[
\begin{array}{c}
1\\
0\\
0\\
\vdots\\
0\\
\end{array}\right],\text{   }
\left[
\begin{array}{c}
0\\
1\\
0\\
\vdots\\
0\\
\end{array}\right],\text{   } \cdots \text{    or    }
\left[
\begin{array}{c}
0\\
0\\
0\\
\vdots\\
1\\
\end{array}\right]\text{ .}
$$

Ví dụ, nếu $x^{(i)}$ là ảnh của chữ số 5, giá trị $y^{(i)}$ tương ứng là một vector 10 chiều với $y_5=1$ và các phần tử khác bằng 0.

### 1.4 Regularized cost function

Hàm cost cho mạng neural với **regularization** được định nghĩa như sau:
$$
\begin{eqnarray}
J(\theta) & = &
\frac{1}{m}\sum_{i=1}^{m}\sum_{k=1}^{K}\left[-y_k^{(i)}\log((h_\theta(x^{(i)}))_k)-(1-y_k^{(i)}) \log(1-(h_\theta(x^{(i)}))_k) \right] + \\
& & \frac{\lambda}{2m} \left[ \sum_{j=1}^{25}\sum_{k=1}^{400}(\Theta_{j,k}^{(1)})^2 +
\sum_{j=1}^{10}\sum_{k=1}^{25}(\Theta_{j,k}^{(2)})^2 \right] & \\
\end{eqnarray}
$$

Các chú ý khi cài đặt cost function.

- Cài đặt nên tổng quát cho số lượng unit bất kỳ của các layer.
- Không thực hiện regularization cho số hạng tương ứng với bias.

Hàm sau sẽ cài đặt regularized cost function. Khi muốn sử dụng phiên bản non-regularizaton, chỉ cần cho giá trị $\lambda=0$.

In [4]:
def sigmoid(z):
    # z can be np.ndarray, np.matrix, or scalar
    return 1 / ( 1 + np.exp(-z) ) 

def nn_cost_function(nn_params, input_layer_size, hidden_layer_size,
                     num_labels, X, y, _lambda):
    """ Tính cost function cho neural networks
    
    Parameters
    -----------
    nn_params : ndarray
             unrolled vector of theta1 and theta2
    theta1 : ndarray
             Tham số của mạng neural cho tầng ẩn (tầng thứ 2)
    theta2 : ndarray
             Tham số của mạng neural cho tầng output (tầng thứ 3)
    input_layer_size : int
    hidden_layer_size : int
    num_labels : int
    X      : ndarray
             Ma trận trong đó mỗi hàng lưu các features của các examples
    y      : ndarray
             Danh sách nhãn của các examples
    _lambda : float
             Tham số trong công thức regularization    
    
    Returns
    -----------
    f : float
        Giá trị của cost function trong mạng neural
    
    Notes
    -----------
    Kích thước của ma trận trong dữ liệu được cung cấp
    theta1 :  25 x 401
    theta2 :  10 x 26
    X : 5000 x 400
    y : (5000, ) ndim = 1
    """    
    m = X.shape[0]
    theta1 = nn_params[0:hidden_layer_size * (input_layer_size + 1)].reshape( 
                             (hidden_layer_size, input_layer_size + 1))    
    
    theta2 = nn_params[hidden_layer_size * (input_layer_size+1):].reshape(
                            (num_labels, hidden_layer_size+1))
        
    X_n = np.concatenate( ( np.ones((m,1)), X ), axis=1 )
    Z2 = np.dot( X_n, theta1.T ) # 5000 x 25
    A2 = sigmoid(Z2)
    A2 = np.concatenate( ( np.ones((m,1)), A2 ), axis=1 )
    Z3 = np.dot( A2, theta2.T ) # 5000 * 10
    A3 = sigmoid(Z3)
    
    # TASK: Cải tiến cài đặt dưới đây -- chỉ dùng 1 vòng lặp trên các examples
    J = 0    
    for i in xrange(m):
        for k in xrange(1, num_labels+1):
            yk = np.int(y[i]==k)
            cost = -yk * np.log( A3[i,k-1] ) - (1-yk) * np.log( 1 - A3[i,k-1] )
            J += cost
    J /= m
    
    # Regularized version of cost function 
    J += _lambda * ( (theta1[:,1:] ** 2).sum() + (theta2[:,1:] ** 2).sum() ) / (2*m)
    
    return J    

Chúng ta sẽ sử dụng hàm ```nn_cost_function``` để tính cost function với dữ liệu và tham số đã cho. Trước đó, chúng ta cần sinh "unrolled" vector gồm các phần tử trong $\Theta_1$ và $\Theta_2$, và khởi tạo các tham số cần thiết cho hàm số.

In [5]:
nn_params = np.concatenate( (Theta1.flatten(), Theta2.flatten()))
input_layer_size  = X.shape[1]
hidden_layer_size = Theta1.shape[0]
num_labels = Theta2.shape[0]

_lambda = 0
J = nn_cost_function(nn_params, input_layer_size, hidden_layer_size, num_labels,
                     X, y, _lambda)
print('Cost at parameters (loaded from ex4weights): %f \n'
      '(this value should be about 0.287629)' % J)

Cost at parameters (loaded from ex4weights): 0.287629 
(this value should be about 0.287629)


Tính cost function với phiên bản regularization.

In [6]:
print 'Checking Cost Function (with Regularization) ...'
_lambda = 1 
J = nn_cost_function(nn_params, input_layer_size, hidden_layer_size, num_labels,
                     X, y, _lambda)
print('Cost at parameters (loaded from ex4weights): %f \n'
      '(this value should be about 0.383770)' % J)

Checking Cost Function (with Regularization) ...
Cost at parameters (loaded from ex4weights): 0.383770 
(this value should be about 0.383770)


## 2. Backpropagation

Trong phần này, chúng ta sẽ cài đặt thuật toán **backpropagation** để tính gradient cho cost function của neural networks.

### 2.1 Sigmoid gradient

Để tính gradient, trước tiên chúng ta sẽ cài đặt hàm sigmoid gradient. Gradient của sigmoid function được tính như sau:

$$g^{\prime}(z)=\frac{d}{dz}g(z)=g(z)(1-g(z))$$

Ở đây $g(z)$ là sigmoid function.

$$sigmoid(z)=g(z)=\frac{1}{1+e^{-z}}$$


In [7]:
def sigmoid_gradient(z):
    # z can be scalar or np.ndarray
    g = sigmoid(z)
    return g * (1-g)

Ta sẽ thử nghiệm hàm đã cài đặt ở trên với một số giá trị của $z$. Khi giá trị rất $z$ rất lớn, hàm số sẽ dần tới 0. Khi $z=0$, hàm số sẽ có giá trị bằng 0.25.

In [8]:
for z in [10000, 0]:
    print 'sigmoid_gradient(%.0f) = %.4f' % (z, sigmoid_gradient(z))
    
g = sigmoid_gradient( np.array([1, -0.5, 0, 0.5, 1]))
print g

sigmoid_gradient(10000) = 0.0000
sigmoid_gradient(0) = 0.2500
[ 0.19661193  0.23500371  0.25        0.23500371  0.19661193]


### 2.2 Random initialization

Khi huấn luyện mạng neural, chúng ta cần khởi tạo các giá trị tham số ban đầu. Khác với thuật toán gradient descent, chúng ta không thể sử dụng zero initialization (nếu sử dụng các tham số tương ứng với các unit ở các tầng sẽ bằng nhau, kết quả là với mỗi ví dụ, mạng neural thu được sẽ trả về xác suất bằng nhau đối với mọi lớp). Để tránh điều này, chúng ta sẽ khởi tạo các tham số ban đầu một cách ngẫu nhiên theo phân bố uniform trong khoảng $[-\epsilon_{init},\epsilon_{init}]$. Chúng ta sẽ chọn $\epsilon_{init}=0.12$. Cài đặt sau sẽ thực hiện công việc đó.

In [9]:
def rand_initialize_weights(L_in, L_out):
    """ Khởi tạo ngẫu nhiên các tham số ban đầu cho một tầng
    Parameters
    -----------
    L_in : int
           Số connection đầu vào của tầng
    L_out : int
           Số connection đầu ra của tầng
           
    Return
    ----------
    W : np.float64
        Ma trận với kích thước L_out x (1 + L_in) gồm các số ngẫu nhiên trong khoảng
        [-epsilon, -epsilon]
    """
    epsilon_init = 0.12
    return np.random.randn(L_out, 1 + L_in) * 2 * epsilon_init - epsilon_init

Sử dụng hàm trên, chúng ta sẽ khởi tạo các giá trị tham số ban đầu cho neural networks.

In [10]:
np.random.seed(99999)
initial_Theta1 = rand_initialize_weights(input_layer_size, hidden_layer_size)
initial_Theta2 = rand_initialize_weights(hidden_layer_size, num_labels)
# unroll parameters
initial_nn_params = np.concatenate( (initial_Theta1.flatten(), 
                                     initial_Theta2.flatten()) )

### 2.3 Backpropagation

<img src="backpropagation_updates.png" alt="Drawing" style="width:400px;"/>

Phần này sẽ hướng dẫn các bạn cài đặt thuật toán **backpropagation**. Ý tưởng chính của thuật toán **backpropagation** như sau. Với một training example $(x^{(t)},y^{(t)})$ cho trước, chúng ta sẽ thực hiện bước "forward pass" để tính tất các giá trị **activation** qua mạng neural, bao gồm cả các giá trị đầu ra của hypothesis $h_\theta(x)$. Sau đó, tại mỗi node $j$ ở tầng $l$, chúng ta sẽ tính "error term" $\delta_j^{(l)}$. "Error term" đánh giá xem một node sẽ chịu đóng góp như thế nào đối với bất kỳ lỗi nào của đầu ra.

Tại các node ở tầng output, chúng ta có thể tính trực tiếp sự khác biệt giữa đầu ra của mạng neural với giá trị thực tế, và sử dụng tính toán đó để định nghĩa $\delta_j^{(3)}$ (bởi vì tầng thứ 3 là tầng output). Với các tầng ẩn, chúng ta sẽ tính $\delta_j^{(l)}$ dựa trên giá trị trung bình trọng số của các "error term" của các node ở tầng $l+1$. Xem mô tả chi tiết thuật toán **backpropagation** tại [ex4.pdf](https://github.com/minhpqn/Machine-Learning-Dojo/blob/master/code/neural_networks_learning/ex4.pdf)

### 2.4 Regularized Neural Networks

Sau khi đã hoàn thành cài đặt thuật toán backpropagation, chúng ta sẽ cài đặt phiên bản regularization cho gradient. Chúng ta thực hiện việc này bằng cách cộng thêm các số hạng bổ sung sau khi tính gradients sử dụng thuật toán backpropagation. 

Đặc biệt, sau khi tính $\Delta_{ij}^{(l)}$ sử dụng thuật toán backpropagation, chúng ta cộng thêm số hạng regularization như sau:

$$\frac{\partial}{\partial \Theta_{ij}^{(l)}} J(\Theta) = D_{ij}^{(l)} = \frac{1}{m} \Delta_{ij}^{(l)} \quad \quad \text{với } j=0$$

$$\frac{\partial}{\partial \Theta_{ij}^{(l)}} J(\Theta) = D_{ij}^{(l)} = \frac{1}{m} \Delta_{ij}^{(l)} + \frac{\lambda}{m}\Theta_{ij}^{(l)} \quad \quad \text{với } j \geq 0$$

Chúng ta sẽ không thực hiện regularization cho cột đầu tiên của $\Theta^{(l)}$ vì nó được sử dụng cho các *bias term*. Hơn thế, trong các tham số $\Theta_{ij}^{(l)}$, chỉ số $i$ bắt đầu từ 1 và $j$ bắt đầu từ 0. Vì thế:

$$\Theta^{(l)}=\left[
\begin{array}{ccc}
\Theta_{1,0}^{(l)} & \Theta_{1,1}^{(l)} & \cdots\\
\Theta_{2,0}^{(l)} & \Theta_{2,1}^{(l)} & \quad\\
\vdots & \quad & \ddots\\
\end{array}
\right]
$$

Code dưới đây sẽ cài đặt thuật toán **backpropagation**.

In [11]:
def nn_gradient(nn_params, input_layer_size, hidden_layer_size, num_labels,
                     X, y, _lambda):
    """ Tính gradient cho cost function trong neural networks
    
    Parameters
    -----------
    nn_params : ndarray
             unrolled vector of theta1 and theta2
    theta1 : ndarray
             Tham số của mạng neural cho tầng ẩn (tầng thứ 2)
    theta2 : ndarray
             Tham số của mạng neural cho tầng output (tầng thứ 3)
             
    input_layer_size : int
    hidden_layer_size : int
    num_labels : int
    
    X      : ndarray
             Ma trận trong đó mỗi hàng lưu các features của các examples
    y      : ndarray
             Danh sách nhãn của các examples
    _lambda : float
             Tham số trong công thức regularization    
    
    Returns
    -----------
    grad : ndarray
        unrolled vector của gradient cho mỗi tầng
    
    Notes
    -----------
    Kích thước của ma trận trong dữ liệu được cung cấp
    theta1 :  25 x 401
    theta2 :  10 x 26
    X : 5000 x 400
    y : (5000, ) ndim = 1
    """    
    
    m = X.shape[0]
    theta1 = nn_params[0:hidden_layer_size * (input_layer_size + 1)].reshape( 
                             (hidden_layer_size, input_layer_size + 1))    
    
    theta2 = nn_params[hidden_layer_size * (input_layer_size + 1):].reshape(
                            (num_labels, hidden_layer_size + 1))
    
    theta1_grad = np.zeros( theta1.shape )
    theta2_grad = np.zeros( theta2.shape )
        
    X_n = np.concatenate( ( np.ones((m,1)), X ), axis=1 )
    Z2 = np.dot( X_n, theta1.T ) # 5000 x 25
    A2 = sigmoid(Z2)
    # A2.shape now -> m x (1+hidden_layer_size)
    A2 = np.concatenate( ( np.ones((m,1)), A2 ), axis=1 )
    Z3 = np.dot( A2, theta2.T ) # 5000 * 10
    A3 = sigmoid(Z3)
    
    c = np.array( range(1, num_labels + 1) )
    for t in xrange(m):
        yt = y[t]
        # Step 1: Perform feedforward pass computing the activations
        # z1, a2, z3, a3 (Already done)
        a1 = X_n[[t],:] # 1 x 401 -- ugly indexing
        z2 = Z2[[t],:]
        a2 = A2[[t],:]
        z3 = Z3[[t],:]
        a3 = A3[[t],:] # shape = 1 x 10        
        
        # Step 2: Compute error terms for output layer
        # For each ouput unit k in layer 3 compute delta3[k]
        
        delta_3 = a3 - ( c == yt ) # shape 1 x 10
        
        # Step 3: Compute error terms for layer 2
        # delta_2 = np.dot(delta_3, theta2) * [ 1 sigmoid_gradient(z2) ]
        sigmoid_grad_z2 = np.concatenate((np.ones( (1,1) ), 
                                         sigmoid_gradient(z2) ),
                                         axis=1)
        delta_2 = np.dot(delta_3, theta2) * sigmoid_grad_z2 # shape = 1 x 26
        
        # Step 4: Accumulate the gradient from this example
        theta1_grad = theta1_grad + np.dot( delta_2[:,1:].T, a1 )
        theta2_grad = theta2_grad + np.dot( delta_3.T, a2 )
        
    theta1_grad /= m
    theta2_grad /= m
    # regularized version for gradient
    theta1_grad[:,1:] += _lambda * theta1[:,1:]/m
    theta2_grad[:,1:] += _lambda * theta2[:,1:]/m
    # unrolled gradients
    grad_ = np.concatenate( ( theta1_grad.flatten(), 
                             theta2_grad.flatten() ) )
      
    return grad_    

### 2.5 Gradient checking

Trong khi huấn luyện một mạng neural, chúng ta sẽ tìm các tham số để cực tiểu hoá hàm cost $J(\Theta)$. Đeer thực hiện gradient checking trên các giá trị tham số, bạn có thể tưởng tượng chúng ta "unroll" các tham số $\Theta^{(1)}$, $\Theta^{(2)}$ thành một vector duy nhất $\theta$. Hàm cost sẽ trở thành hàm $J(\theta)$ của **unrolled** vector. Nhờ đó chúng ta có thể thực hiện gradient checking như sau.

Giả sử chúng ta có hàm số $f_i(\theta)$ để tính giá trị $\frac{\partial}{\partial \theta_i}J(\theta)$ và bạn muốn kiểm tra xem liệu $f_i$ có trả về các giá trị đạo hàm chính xác hay không.

$$\text{Đặt}\quad
\theta^{(i+)}=\theta+
\left[
\begin{array}{c}
0\\
0\\
\vdots\\
\epsilon\\
\vdots\\
0\\
\end{array}\right]
\quad\text{ và }\quad
\theta^{(i+)}=\theta-
\left[
\begin{array}{c}
0\\
0\\
\vdots\\
\epsilon\\
\vdots\\
0\\
\end{array}\right]
$$

$\theta^{(i+)}$ giống hệt $\theta$ ngoại trừ phần tử thứ $i$-th được cộng thêm $\epsilon$. Tương tự như vậy $\theta^{(i-)}$ là vector với phần tử thứ $i$-th bị giảm đi $\epsilon$. Bây giờ bạn có thể đánh giá tính chính xác $f_i(\theta$ bằng cách kiểm tra, liệu với mỗi giá trị của $i$, điều kiện sau có thoả mãn không:

$$f_i(\theta)\approx \frac{J(\theta^{(i+)}) - J(\theta^{(i-)})}{2\epsilon}$$

Mức độ xấp xỉ của hai giá trị trong công thức trên phụ thuộc vào hàm cost $J$. Nhưng giả sử $\epsilon=10^{-4}$, bạn sẽ luôn thấy vế trái và vế phải trong công thức trên sẽ giống nhau tới 4 chữ số sau phần thập phân (thường là nhiều hơn).

Đoạn code dưới đây sẽ cài đặt numerical gradient.

In [12]:
def compute_numerical_gradient(J, theta):
    """ Tính numerical gradient theo công thức trên
    
    Parameters
    --------------
    J : wrapper function cho cost function. Hàm này nhận đối số là tham số theta
    theta: numpy.ndarray
        unrolled vector chứa các tham số của mạng neural
        
    Returns
    -------------
    numgrad : numpy.ndarray
       Giá trị của hàm gradient (numerical version)
    """
    numgrad = np.zeros( theta.shape )    
    perturb = np.zeros( theta.shape )
    e = 1e-4
    for p in range( len(theta) ):
        # Set perturbation vector
        perturb[p] = e
        loss1 = J(theta - perturb)
        loss2 = J(theta + perturb)
        # Compute numerical gradient
        numgrad[p] = (loss2 - loss1) / (2*e)
        perturb[p] = 0
        
    return numgrad

Hàm dưới đây sẽ tạo ra một mạng neural với kích thước nhỏ và kiểm tra các giá trị backpropagation gradients.

In [13]:
def debugInitializeWeights(fan_out, fan_in):
    """
    initializes the weights 
    of a layer with fan_in incoming connections and fan_out outgoing 
    connections using a fix set of values
    Note that W should be set to a matrix of size(1 + fan_in, fan_out) as
    the first row of W handles the "bias" terms
    """
    W = np.zeros( (fan_out, 1 + fan_in) )
    W = np.sin( range(1, W.size+1) ).reshape(W.shape)/10
    return W  
    
def checkNNGradients(_lambda=0):
    """ Tạo một mạng neural cỡ nhỏ để kiểm tra backpropagation gradients
    Returns
    -------------
    None
    """
    
    input_layer_size = 3
    hidden_layer_size = 5
    num_labels = 3
    m = 5
    theta1 = debugInitializeWeights(hidden_layer_size, input_layer_size)
    theta2 = debugInitializeWeights(num_labels, hidden_layer_size)

    X = debugInitializeWeights(m, input_layer_size -1)
    y = 1 + np.mod( range(1,m+1), num_labels)
    nn_params = np.concatenate( ( theta1.flatten(), theta2.flatten()) )
    costFunc = lambda p: nn_cost_function(p, input_layer_size, 
                                          hidden_layer_size, num_labels,
                                          X, y, _lambda)
    
    gradFunc = lambda p: nn_gradient(p, input_layer_size, 
                                     hidden_layer_size, num_labels,
                                     X, y, _lambda)
    
    grad = gradFunc(nn_params)
    numgrad = compute_numerical_gradient(costFunc, nn_params)
      
    print 'The above two columns you get should be very similar.'
    print '(Left-Your Numerical Gradient, Right-Analytical Gradient)\n'
    
    # Evaluate the norm of the difference between two solutions.
    # If you have a correct implementation, and assuming you used 
    # EPSILON = 0.0001 in compute_mumerical_gradient(),
    # then diff below should be less than 1e-9
    
    diff = np.linalg.norm(numgrad-grad)/np.linalg.norm(numgrad+grad)
    
    print 'If your backpropagation implementation is correct, then'
    print 'the relative difference will be small (less than 1e-9).'
    print 'Relative Difference: %g' % diff    

Ta sẽ sử dụng hàm cài đặt ở trên để kiểm tra xem cài đặt thuật toán backpropagation gradients đã chính xác chưa.

In [14]:
print 'Checking Backpropagation...'
checkNNGradients()
print '\nChecking Backpropagation (w/ Regularization)'
_lambda = 3
checkNNGradients(_lambda)
debug_J = nn_cost_function(nn_params, input_layer_size,
                          hidden_layer_size, num_labels, X, y, _lambda);

print('\n\nCost at (fixed) debugging parameters (w/ lambda = 3): %f\n'
      '\n(this value should be about 0.576051)\n' % debug_J);

Checking Backpropagation...
The above two columns you get should be very similar.
(Left-Your Numerical Gradient, Right-Analytical Gradient)

If your backpropagation implementation is correct, then
the relative difference will be small (less than 1e-9).
Relative Difference: 2.07115e-11

Checking Backpropagation (w/ Regularization)
The above two columns you get should be very similar.
(Left-Your Numerical Gradient, Right-Analytical Gradient)

If your backpropagation implementation is correct, then
the relative difference will be small (less than 1e-9).
Relative Difference: 2.025e-11


Cost at (fixed) debugging parameters (w/ lambda = 3): 0.576051

(this value should be about 0.576051)



### 2.6 Học tham số với fmin_l_bfgs_b

Sau khi cài đặt xong hàm cost và gradient cho mạng neural, trong phần tiếp theo, chúng ta sẽ học các tham số sử dụng thuật toán tối ưu hoá L-BFGS-B. Trong python, chúng ta sẽ dùng hàm [fmin_l_bfgs_b](http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.optimize.fmin_l_bfgs_b.html).

Sau khi huấn luyện xong mạng neural, tôi sẽ tính độ chính xác của dự đoán trên dữ liệu.

In [15]:
def predict(Theta1, Theta2, X):
    m = X.shape[0]
    X_new = np.concatenate( (np.ones( (m,1) ), X), axis=1 ) # 5000 x 401
    A2 = sigmoid( np.dot(X_new, Theta1.T) ) # 5000 x 25
    A2 = np.concatenate( ( np.ones( (m,1) ), A2 ), axis=1 ) # 5000 x 26
    A3 = sigmoid( np.dot(A2, Theta2.T) ) # 5000 x 10
    p = np.argmax( A3, axis=1 ) + 1
    return p

from scipy.optimize import fmin_l_bfgs_b
_lambda = 1
input_layer_size  = 400
hidden_layer_size = 25
num_labels = 10

print 'Training Neural Network...'
nn_params, f, d = fmin_l_bfgs_b(nn_cost_function, initial_nn_params,
                          fprime=nn_gradient, 
                          args=(input_layer_size, 
                                hidden_layer_size, 
                                num_labels,
                                X, y, _lambda),
                          disp=False, maxiter=50)

Theta1 = nn_params[0:hidden_layer_size * (input_layer_size + 1)].reshape(
                           (hidden_layer_size, input_layer_size + 1))    
Theta2 = nn_params[hidden_layer_size * (input_layer_size + 1):].reshape(
                            (num_labels, hidden_layer_size + 1))

pred = predict(Theta1, Theta2, X)
print('\nTraining accuracy (with neural networks): %s' % 
      ( 100 * (pred == y).mean() ) )

Training Neural Network...

Training accuracy (with neural networks): 95.58


Còn 3 phần trong bài tập này tôi chưa hoàn thành:

- Visualizing data
- Hiển thị tầng ẩn
- Thay đổi các tham số $\lambda$ và số lượng vòng lặp và quan sát sự thay đổi của kết quả thu được.

Bạn có thể xem cài đặt chi tiết của bài tập này trong một script duy nhất tại [đây](https://github.com/minhpqn/Machine-Learning-Dojo/blob/master/code/neural_networks_learning/Neural_Networks_Learning.py).